In [1]:
import math
import pandas as pd

df=pd.read_csv("id3.csv")
print('Input Data Set is : \n',df)

t=df.keys()[-1]
print("Target Attribute is:", t)
attribute_names = list(df.keys())
attribute_names.remove(t)
print("predicting Attributes:", attribute_names)

def entrophy(probs):
    return sum([-prob * math.log(prob,2) for prob in probs])

def entrophy_of_list(ls,value):
    from collections import Counter
    cnt=Counter(x for x in ls)
    print("Target attribute class count(Yes/NO)=", dict(cnt))
    total_instances=len(ls)
    print("Total no of instances/records associated with {0} is : {1}". format(value, total_instances))
    probs = [x/total_instances for x in cnt.values()]
    return entrophy(probs)

def information_gain(df,split_attribute,target_attribute,battr):
    print("\n\n------Information Gain Calculation of", split_attribute,"---")
    df_split =df.groupby(split_attribute)
    glist=[]
    
    for gname,group in df_split:
        print("Grouped Attribute Values\n",group)
        glist.append(gname)
    nobs=len(df.index)
    df_agg1=df_split.agg({target_attribute:lambda x:entrophy_of_list(x,glist.pop())})
    df_agg2=df_split.agg({target_attribute:lambda x:len(x)/nobs})
    
    df_agg1.columns = ['Entrophy']
    df_agg2.columns = ['Proportion']
    
    new_entrophy = sum(df_agg1['Entrophy']*df_agg2['Proportion'])
    
    if battr!='S':
        old_entrophy = entrophy_of_list(df[target_attribute],'S-' + df.iloc [0] [df.columns.get_loc(battr)])
    else:
        old_entrophy = entrophy_of_list(df[target_attribute],battr)
        
    return old_entrophy - new_entrophy

def id3(df,target_attribute,attribute_names,default_class=None, default_attr='S'):
    from collections import Counter
    cnt=Counter(x for x in df[target_attribute])
    if len(cnt)==1:
        return next(iter (cnt))
    elif df.empty or(not attribute_names):
        return default_class
    else:
        default_class = max(cnt.keys())
        gainx = []
        for attr in attribute_names:
            ig= information_gain (df, attr, target_attribute,default_attr)
            gainx.append(ig)
            print("Information gain of",attr ,"is:",ig)
        index_of_max=gainx.index(max(gainx))
        best_attr=attribute_names[index_of_max]
        print("\n Attribute with the maximum gain is:",best_attr)
        tree={best_attr :{}}
        remaining_atribute_names = [i for i in attribute_names if i!=best_attr]
        
        for attr_val, data_subset in df.groupby(best_attr):
            subtree = id3(data_subset,target_attribute,remaining_atribute_names,default_class,best_attr)
            tree[best_attr][attr_val]= subtree
        return tree
                            
from pprint import pprint
tree=id3(df,t,attribute_names)
print("\n The Resultant Tree is:")
pprint(tree)

Input Data Set is : 
      Outlook Temperature Humidity    Wind Answer
0      sunny         hot     high    weak     no
1      sunny         hot     high  strong     no
2   overcast         hot     high    weak    yes
3       rain        mild     high    weak    yes
4       rain        cool   normal    weak    yes
5       rain        cool   normal  strong     no
6   overcast        cool   normal  strong    yes
7      sunny        mild     high    weak     no
8      sunny        cool   normal    weak    yes
9       rain        mild   normal    weak    yes
10     sunny        mild   normal  strong    yes
11  overcast        mild     high  strong    yes
12  overcast         hot   normal    weak    yes
13      rain        mild     high  strong     no
Target Attribute is: Answer
predicting Attributes: ['Outlook', 'Temperature', 'Humidity', 'Wind']


------Information Gain Calculation of Outlook ---
Grouped Attribute Values
      Outlook Temperature Humidity    Wind Answer
2   overcast      